<a href="https://colab.research.google.com/github/RoyEHamlin/Websockets/blob/main/websockets_v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Cryptocurrency feed from Binaance using Websockets
## Tutorial at : https://www.youtube.com/watch?v=z2ePTq-KTzQ
## Resources :
### https://pypi.org/project/websocket-client/
### https://websocket-client.readthedocs.io/en/latest/
### https://binance-docs.github.io/apidocs/futures/en/
### https://developers.binance.com/docs/binance-api/spot-detail/web-socket-streams/#klinecandlestick-streams (example)
### https://coinderivs.ams3.cdn.digitaloceanspaces.com/recent_liquidations.json (Binance)
### https://fapi.binance.com/fapi/v1/depth?symbol=BTCUSDT&limit=1000 (orderbook)

In [ ]:
pip install websocket-client

     |████████████████████████████████| 53 kB 880 kB/s 


In [ ]:
import websocket, json

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
cc = 'btcusd'
interval = '1m'

socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

In [ ]:
import pandas as pd
from datetime import datetime
import time

r1 = {
    "unix_time" : 1.1,
    "symbol" : "BTCUSDT",
    "open" : 1.2,
    "close" : 1.3,
    "high" : 1.4,
    "low" : 1.5,
    "vol" : 1.6,
    "buy_vol" : 1.7,
    "num_trades" : 1,
    "candle_close" : True
}

# creates the df, then drops the dummy row to cast columns as correct types ONLY.
df = pd.DataFrame(r1, index=['a'])
df = df.drop('a') # d
old_time = int(datetime.fromtimestamp(time.time()).strftime("%S")) // 5 # old time stamp, as 1/12 of a minute
new_time = int(datetime.fromtimestamp(time.time()).strftime("%S")) // 5 # new time stamp, as 1/12 of a minute

In [ ]:
# from numpy.ma.extras import count_masked

def on_message(ws, message):
    msg = json.loads(message)

    # if msg is next multiple of "5", save this row.  Otherwise ignore.
    global old_time
    global new_time
    print(old_time)
    # print(int(datetime.fromtimestamp(msg["E"]).strftime("%S")))
    # print(int(datetime.fromtimestamp(msg["E"]/1000).strftime("%S")))
    new_time = int(datetime.fromtimestamp(msg["E"]/1000).strftime("%S")) // 5
    # print(new_time)
    # print(f"old:{old_time} and new:{new_time}")
    if old_time != new_time:
        old_time = new_time
    
        row = {
            "unix_time" : msg["E"]/1000,
            "symbol" : msg["k"]["s"],
            "open" : msg["k"]["o"],
            "close" : msg["k"]["c"],
            "high" : msg["k"]["h"],
            "low" : msg["k"]["l"],
            "vol" : msg["k"]["v"],
            "buy_vol" : msg["k"]["V"],
            "num_trades" : int(msg["k"]["n"]),
            "candle_close" : bool(msg["k"]["x"])
        }

        global df
        df = df.append(row, ignore_index=True)

        # df.to_csv("btcprices.csv", index=False, encoding='utf-8-sig')

        print(row)
        with open('/gdrive/MyDrive/crypto/btcusdt.csv', 'w') as writefile:
            writefile.write(row)





def on_close(ws):
    print("Connection Closed")

def on_open(ws):
    def run(*args):
        for i in range(3):
            time.sleep(1)
            ws.send("Hello %d" % i)
        time.sleep(1)
        ws.close()
        print("thread terminating...")
    _thread.start_new_thread(run, ())



ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close)

In [ ]:
ws.run_forever()


9
{'unix_time': 1639780913.045, 'symbol': 'BTCUSDT', 'open': '46298.59000000', 'close': '46296.18000000', 'high': '46298.60000000', 'low': '46295.00000000', 'vol': '10.00030000', 'buy_vol': '7.82463000', 'num_trades': 355, 'candle_close': False}
10
{'unix_time': 1639780915.115, 'symbol': 'BTCUSDT', 'open': '46298.59000000', 'close': '46296.18000000', 'high': '46298.60000000', 'low': '46295.00000000', 'vol': '10.02332000', 'buy_vol': '7.84688000', 'num_trades': 358, 'candle_close': False}
11
11
11
{'unix_time': 1639780920.001, 'symbol': 'BTCUSDT', 'open': '46298.59000000', 'close': '46301.39000000', 'high': '46309.26000000', 'low': '46295.00000000', 'vol': '10.98056000', 'buy_vol': '8.67580000', 'num_trades': 406, 'candle_close': True}
0
0
0
{'unix_time': 1639780927.036, 'symbol': 'BTCUSDT', 'open': '46301.38000000', 'close': '46300.37000000', 'high': '46301.39000000', 'low': '46295.12000000', 'vol': '1.02437000', 'buy_vol': '0.36406000', 'num_trades': 35, 'candle_close': False}
1
1
{'u

False

In [ ]:
 # https://towardsdatascience.com/google-colab-import-and-export-datasets-eccf801e2971
 df.to_csv("/btcprices.csv", index=False, encoding='utf-8-sig')
 df
#  from google.colab import files
#  df.to_csv('sample.csv')
#  files.download('sample.csv')

,unix_time,symbol,open,close,high,low,vol,buy_vol,num_trades,candle_close
0,1.639781e+09,BTCUSDT,46298.59000000,46296.18000000,46298.60000000,46295.00000000,10.00030000,7.82463000,355,False
1,1.639781e+09,BTCUSDT,46298.59000000,46296.18000000,46298.60000000,46295.00000000,10.02332000,7.84688000,358,False
2,1.639781e+09,BTCUSDT,46298.59000000,46301.39000000,46309.26000000,46295.00000000,10.98056000,8.67580000,406,True
3,1.639781e+09,BTCUSDT,46301.38000000,46300.37000000,46301.39000000,46295.12000000,1.02437000,0.36406000,35,False
4,1.639781e+09,BTCUSDT,46301.38000000,46297.12000000,46301.39000000,46295.01000000,1.75706000,0.49834000,76,False
5,1.639781e+09,BTCUSDT,46301.38000000,46295.77000000,46301.39000000,46295.01000000,2.06654000,0.56894000,99,False
6,1.639781e+09,BTCUSDT,46301.38000000,46295.90000000,46301.39000000,46295.01000000,2.60764000,0.87445000,119,False
7,1.639781e+09,BTCUSDT,46301.38000000,46290.65000000,46301.39000000,46284.53000000,4.73288000,1.57457000,198,False
8,1.639781e+09,BTCUSDT,46301.38000000,46281.74000000,46301.39000000,46280.03000000,5.79392000,2.33997000,269,False
9,1.639781e+09,BTCUSDT,46301.38000000,46280.02000000,46301.39000000,46280.00000000,6.08705000,2.49097000,296,False


In [ ]:
with open('/btcusdt.csv', 'w') as writefile:
    writefile.write("test\ttest2\n")